In [ ]:
from google.colab import drive
drive.mount('/content/drive')



ValueError: mount failed

In [ ]:
data_path = '/content/drive/My Drive/solar_panel/dataset'


import os

SAVE_DIR = "/content/drive/My Drive/solar_panel/dataset"
os.makedirs(SAVE_DIR, exist_ok=True)

In [ ]:
!git clone https://github.com/facebookresearch/sam2.git
%cd sam2
!pip install -e .
!pip install opencv-python matplotlib tqdm
%cd ..


In [ ]:
import sys
sys.path.append('/content/sam2')

from sam2.build_sam import build_sam2
import torch

device = 'cuda' if torch.cuda.is_available() else 'cpu'
model_cfg = 'sam2_hiera_s'
sam_model = build_sam2(model_cfg, checkpoint=None)
sam_model.to(device)
print("SAM2 initialized on", device)


#Creating Dataset and DataLoaders

In [ ]:
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as T
import torch
import cv2, os, numpy as np

class SolarSegmentationDataset(Dataset):
    def __init__(self, img_dir, mask_dir, transform=None):
        self.img_dir = img_dir
        self.mask_dir = mask_dir
        self.images = sorted(os.listdir(img_dir))
        self.masks = sorted(os.listdir(mask_dir))
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.images[idx])
        mask_path = os.path.join(self.mask_dir, self.masks[idx])

        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)

        img = cv2.resize(img, (400, 400))
        mask = cv2.resize(mask, (400, 400))
        mask = (mask > 128).astype(np.float32)

        if self.transform:
            img = self.transform(img)
        mask = torch.tensor(mask).unsqueeze(0)

        return img, mask

# Normalization and Tensor conversion
class SegmentationTransform:
    def __init__(self):
        self.transforms = T.Compose([
            T.ToTensor(),
            T.Normalize(mean=[0.485, 0.456, 0.406],
                        std=[0.229, 0.224, 0.225])
        ])
    def __call__(self, img):
        return self.transforms(img)

train_dataset = SolarSegmentationDataset(
    img_dir='/content/drive/MyDrive/solar_panel/dataset/train/img',
    mask_dir='/content/drive/MyDrive/solar_panel/dataset/train/mask',
    transform=SegmentationTransform()
)

val_dataset = SolarSegmentationDataset(
    img_dir='/content/drive/MyDrive/solar_panel/dataset/val/img',
    mask_dir='/content/drive/MyDrive/solar_panel/dataset/val/mask',
    transform=SegmentationTransform()
)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False, num_workers=2)


# Zero Shot

In [ ]:
from sam2.sam2_image_predictor import SAM2ImagePredictor
from sam2.build_sam import build_sam2

device = 'cuda' if torch.cuda.is_available() else 'cpu'
cfg_path = "sam2_hiera_s.yaml"

checkpoint = None
sam_model = build_sam2(cfg_path, checkpoint)
predictor = SAM2ImagePredictor(sam_model)
predictor.model.to(device)
predictor.model.eval()

# Loading test image
import cv2
import matplotlib.pyplot as plt

test_img_path = '/content/drive/MyDrive/solar_panel/dataset/val/img/WTPSJ54CCFQAAS.png'  # <-- change this to any val image
image = cv2.cvtColor(cv2.imread(test_img_path), cv2.COLOR_BGR2RGB)
predictor.set_image(image)

# Generating mask
masks, scores, logits = predictor.predict(point_coords=None, point_labels=None, multimask_output=True)

print(f"Generated {len(masks)} masks")
plt.figure(figsize=(15, 5))

for i in range(3):
    plt.subplot(1, 3, i+1)
    plt.imshow(masks[i], alpha=0.6, cmap='jet')  # overlay mask
    plt.title(f"Mask {i+1}\nConfidence: {scores[i]:.2f}")
    plt.axis('off')

plt.tight_layout()
plt.show()


In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import os


sample_files = [
    "ZLVSC7C2CEKGIW.png",
    "STFNN6D12IUTJU.png",
    "IJYJF237BDLZX.png"
]

val_img_dir = "/content/drive/MyDrive/solar_panel/dataset/val/img"
val_mask_dir = "/content/drive/MyDrive/solar_panel/dataset/val/mask"

for name in sample_files:
    img_path = os.path.join(val_img_dir, name)
    mask_path = os.path.join(val_mask_dir, name)

    # Loading RGB image and ground truth mask
    image = cv2.cvtColor(cv2.imread(img_path), cv2.COLOR_BGR2RGB)
    gt_mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
    gt_mask = cv2.resize(gt_mask, (400, 400))
    gt_mask = (gt_mask > 128).astype(np.uint8)

    # Predicting masks (zero-shot)
    predictor.set_image(image)
    masks, scores, _ = predictor.predict(point_coords=None, point_labels=None, multimask_output=True)

    # Highest-confidence mask
    best_mask = masks[np.argmax(scores)].astype(np.uint8)

    # Plot
    plt.figure(figsize=(12, 4))
    plt.subplot(1, 3, 1)
    plt.imshow(image)
    plt.title("Original Image")
    plt.axis("off")

    plt.subplot(1, 3, 2)
    plt.imshow(gt_mask, cmap="gray")
    plt.title("Ground Truth")
    plt.axis("off")

    plt.subplot(1, 3, 3)
    plt.imshow(image)
    plt.imshow(best_mask, cmap="jet", alpha=0.4)
    plt.title(f"SAM2 Prediction (score={scores.max():.2f})")
    plt.axis("off")

    plt.suptitle(name)
    plt.show()


# One Shot

In [ ]:
from sam2.build_sam import build_sam2
from sam2.sam2_image_predictor import SAM2ImagePredictor
import torch, cv2, numpy as np
from matplotlib import pyplot as plt
import os

device = "cuda" if torch.cuda.is_available() else "cpu"
cfg_path = "/content/sam2/sam2/configs/sam2_hiera_s.yaml"
checkpoint = "/content/sam2/checkpoints/sam2_hiera_s.pth"

sam_model = build_sam2(cfg_path, checkpoint)
predictor = SAM2ImagePredictor(sam_model)
predictor.model.to(device)


In [ ]:

img_path = "/content/drive/MyDrive/solar_panel/dataset/val/img/WUSAY7147GDFYZ.png"
mask_path = "/content/drive/MyDrive/solar_panel/dataset/val/mask/WUSAY7147GDFYZ.png"

image = cv2.imread(img_path)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
gt_mask = cv2.imread(mask_path, 0)  # grayscale


In [ ]:
import cv2
import matplotlib.pyplot as plt


image_name = "XHZOH4134EPSDU.png"
img_path = f"/content/drive/MyDrive/solar_panel/dataset/val/img/{image_name}"


image = cv2.cvtColor(cv2.imread(img_path), cv2.COLOR_BGR2RGB)


point = [200, 205]
label = [1]  # 1 = foreground (panel), 0 = background



image_dot = image.copy()
cv2.drawMarker(image_dot, tuple(point), (255, 0, 0), markerType=cv2.MARKER_STAR, markerSize=20, thickness=3)



plt.figure(figsize=(6, 6))
plt.imshow(image_dot)
plt.title(f"Prompt point for {image_name} at {point}")
plt.axis("off")
plt.show()


In [ ]:
import numpy as np

predictor.set_image(image)


point_coords = np.array([point])
point_labels = np.array(label)


masks, scores, _ = predictor.predict(
    point_coords=point_coords,
    point_labels=point_labels,
    multimask_output=True
)


best_mask = masks[np.argmax(scores)]

# Visualize
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.imshow(image)
plt.scatter(point[0], point[1], color='red', marker='*', s=200)
plt.title("Prompt Point")
plt.axis("off")

plt.subplot(1, 2, 2)
plt.imshow(image)
plt.imshow(best_mask, cmap="jet", alpha=0.5)
plt.scatter(point[0], point[1], color='red', marker='*', s=200)
plt.title(f"SAM2 Segmentation (score={scores.max():.2f})")
plt.axis("off")

plt.show()


# Fine Tuning

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os
SAVE_DIR = "/content/drive/My Drive/solar_panel/dataset"
os.makedirs(SAVE_DIR, exist_ok=True)

# Clone SAM2 repo and install
!git clone https://github.com/facebookresearch/sam2.git
%cd sam2
!pip install -e .
!pip install opencv-python matplotlib tqdm
%cd /content/sam2

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
fatal: destination path 'sam2' already exists and is not an empty directory.
/content/sam2
Obtaining file:///content/sam2
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for SAM-2 (pyproject.toml) ... done
  Created wheel for SAM-2: filename=sam_2-1.0-0.editable-cp312-cp312-linux_x86_64.whl size=13852 sha256=66441ba57a1dafebbcc98ab5113f6cdb801bdf4ac89f4f934adcfd0c07252843
  Stored in directory: /tmp/pip-ephem-wheel-cache-tgcdmm6f/wheels/9e/fa/17/14aaeb20d3ca07c58ee93742054d4479f89c243063ce0b61b9
Successfully built SAM-2
  Attempting uninstall: SAM-2
    Found existing installation: SAM-2 1.0
    Uninstalling SAM-2-1.0:
      Successfully uninstalled SAM-2-1.0


/content/sam2


In [ ]:
import sys
sys.path.append('/content/sam2/sam2')
from build_sam import build_sam2
from sam2_image_predictor import SAM2ImagePredictor
import torch



In [ ]:
!git clone https://github.com/facebookresearch/sam2.git
%cd sam2


Cloning into 'sam2'...
remote: Enumerating objects: 1070, done.
remote: Total 1070 (delta 0), reused 0 (delta 0), pack-reused 1070 (from 1)
Receiving objects: 100% (1070/1070), 128.11 MiB | 15.08 MiB/s, done.
Resolving deltas: 100% (380/380), done.
/content/sam2


In [ ]:
!pip install -e .


In [ ]:
%cd checkpoints
!chmod +x download_ckpts.sh
!./download_ckpts.sh
